In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# ■ 大学教育付加価値（Value-Added）検証のためのダミーデータ生成モデル
# 目的: 入学時学力(Input)だけでなく、生活習慣や学習方略(Process)がGPA(Output)に与える影響を検証する
# 前提: I-E-Oモデル (Input-Environment-Output) に基づく変数設定

np.random.seed(42)  # 再現性の確保
N = 3500            # サンプルサイズ（学生数）

# ---------------------------------------------------------
# 1. データ生成 (Data Generation)
# ---------------------------------------------------------

# --- Input: 入学時特性 ---
# 入学時学力スコア (偏差値基準: Mean=55, SD=10)
input_score = np.random.normal(55, 10, N)

# --- Environment: 生活・学習習慣 ---
# 睡眠時間 (Mean=6.5h)
sleep_hours = np.random.normal(6.5, 1.0, N)

# 運動習慣 (0=なし, 1=あり) と強度
exercise_flag = np.random.binomial(1, 0.4, N)
exercise_intensity = exercise_flag * np.random.normal(2, 1, N)

# 学習方略 (Study Strategy): 睡眠と運動が良いと方略も改善する仮定
# ※ 深い学習(Deep Approach)の程度を数値化
study_strategy = (0.4 * sleep_hours) + (0.3 * exercise_intensity) + np.random.normal(0, 1, N)

# 学習時間 (Study Hours): 方略が良い学生は時間も長い傾向
study_hours = (1.2 * study_strategy) + np.random.normal(0, 1, N)

# アルバイト時間 (Work Hours): 経済的制約の代替指標 (0-40時間でクリップ)
work_hours = np.clip(np.random.normal(12, 8, N), 0, 40)

# --- Output: GPAの算出モデル ---
# 各変数の重み付け（係数）を設定
# ・入学時学力の影響は限定的 (0.25)
# ・学習方略（質）の影響を最大化 (0.35)
# ・バイト時間が長すぎるとマイナス効果 (-0.25)
raw_gpa = (
    0.25 * ((input_score - 55) / 10) +       # 標準化して加算
    0.35 * ((study_strategy - np.mean(study_strategy)) / np.std(study_strategy)) +
    0.10 * ((study_hours - np.mean(study_hours)) / np.std(study_hours)) -
    0.25 * (work_hours / 40 * 2) +           # バイトの負の影響
    2.8 +                                    # ベースラインGPA
    np.random.normal(0, 0.3, N)              # 誤差項
)

# GPAを 0.0 - 4.0 の範囲に収める
gpa_final = np.clip(raw_gpa, 0.0, 4.0)

# データフレーム化
df = pd.DataFrame({
    "Input_Score": input_score,
    "Sleep_Hours": sleep_hours,
    "Study_Strategy": study_strategy,
    "Study_Hours": study_hours,
    "Work_Hours": work_hours,
    "GPA": gpa_final
})

# CSV保存（実務ではこれを分析に回す）
# df.to_csv("student_value_added_data.csv", index=False)
print("=== データ生成完了 (先頭5行) ===")
display(df.head())

# ---------------------------------------------------------
# 2. 簡易分析: 重回帰モデル (OLS Analysis)
# ---------------------------------------------------------
# 説明変数 (X) と 目的変数 (y)
X = df[["Input_Score", "Sleep_Hours", "Study_Strategy", "Study_Hours", "Work_Hours"]]
X = sm.add_constant(X) # 定数項（切片）の追加
y = df["GPA"]

# モデル構築と学習
model = sm.OLS(y, X).fit()

# 結果の表示
print("\n=== 重回帰分析結果 (GPA決定要因) ===")
print(model.summary())

=== データ生成完了 (先頭5行) ===


,Input_Score,Sleep_Hours,Study_Strategy,Study_Hours,Work_Hours,GPA
0,59.967142,7.680641,2.643074,4.216215,5.864470,2.771957
1,53.617357,5.872687,2.429070,3.060880,7.792378,2.641161
2,61.476885,6.545223,2.739592,4.686608,14.067681,2.825120
3,70.230299,6.551198,1.664725,2.618500,9.388139,2.908395
4,52.658466,5.998216,2.000585,1.782905,30.492875,1.768149



=== 重回帰分析結果 (GPA決定要因) ===
                            OLS Regression Results                            
Dep. Variable:                    GPA   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     2054.
Date:                Sat, 29 Nov 2025   Prob (F-statistic):               0.00
Time:                        22:50:36   Log-Likelihood:                -652.38
No. Observations:                3500   AIC:                             1317.
Df Residuals:                    3494   BIC:                             1354.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            